# A python solution to calculate Pv for pure substances using PR-EOS

***
**Note: Click on the `Cell` in the task bar above, then click on `Run All` to run the entire code**


In [1]:
import ipywidgets as widgets
import pandas as pd
import numpy as np
from IPython.display import display, HTML
import math

## The cell below solves question 1
Calculate the vapor pressure of propane together with the densities of the liquid and gas at 104°F.
Compare your answers with values from Figures 2-7, 2-12, and 3-4 (McCain Jr., W.D. The
Properties of Petroleum Fluids (2nd Edition). PennWell Publishing Company, Tulsa, OK, 1990.). 

In [2]:
# Create some widgets
title = widgets.HTML(value="<h2>Select the species and input the temperature</h2>")
temperature = widgets.FloatText(value="104", description="Farenheit")
pressure_widget = widgets.FloatText(value="100", min=14.7, description="Starting Pressure (psia)")


# working data
species_list = {
    "1": { "name": "iso-butane", "pc": 527.9, "tc": 274.46, "w": 0.1852 },
    "2": { "name": "propane", "pc": 616.0, "tc": 206.06, "w": 0.1522 }
}
# loop through to key the dropdown options
dropdown_options = [(value["name"], int(key)) for key, value in species_list.items()]

# specie dropdown
specie_dropdown = widgets.Dropdown(
    options=dropdown_options,
    value=2,
    description='Specie',
)

# interact with widgets
def on_value_change(sys_temp, specie, initial_trial_pressure):
    selected_specie = species_list[f"{specie}"]
    pc = selected_specie['pc']
    tc = selected_specie['tc']
    tr = round(calculate_tr(sys_temp, tc), 5)
    b = round(calculate_b(tc, pc), 4)
    ac = round(calculate_ac(tc, pc), 0)
    w = selected_specie['w']
    alpha = round(calculate_alpha(w, sys_temp, tc), 4)
    aT = round(calculate_at(ac, alpha), 0)
    
    initial_data = {
        "Tr": [tr],
        "b": [b],
        "ac": [ac],
        "alpha": [alpha],
        "at": [aT]
    }
    
    df = pd.DataFrame(initial_data)
    df = df.transpose()
    display(df)
    
    display(HTML(f"<h2 style='color:green'>ITERATION DISPLAYED ON TABLE</h2>"))
    
    iteration_table_data = {
        "z1": [],
        "z2": [],
        "z3": [],
        "zg": [],
        "zl": [],
        "fg": [],
        "fl": [],
        "fg - fl": [],
        "pressure": [],
        "Vmg": [],
        "Vml": [],
    }
    
    fg = 0
    fl = 1000
    pressure = initial_trial_pressure
    tolerance = 0.1
    while abs(fg - fl) >= tolerance:
        A = round(calculate_A(aT, pressure, sys_temp), 5)
        B = round(calculate_B(b, pressure, sys_temp), 6)
        roots = cubic_root_calculation(A, B)

        iteration_table_data["z1"].append(roots[0])
        iteration_table_data["z2"].append(roots[1])
        iteration_table_data["z3"].append(roots[2])

        z_values = get_highest_and_lowest_root(roots)
        iteration_table_data["zg"].append(z_values["zg"])
        iteration_table_data["zl"].append(z_values["zl"])

        fg = solve_for_fg_fl(z_values["zg"], B, A, pressure)
        fl = solve_for_fg_fl(z_values["zl"], B, A, pressure)
        iteration_table_data["fg"].append(fg)
        iteration_table_data["fl"].append(fl)
        
        iteration_table_data["pressure"].append(pressure)
        iteration_table_data["fg - fl"].append(abs(fg - fl))
        
        Vmg = (z_values["zg"] * 10.732 * convert_f_to_r(sys_temp)) / pressure
        Vml = (z_values["zl"] * 10.732 * convert_f_to_r(sys_temp)) / pressure
        iteration_table_data["Vmg"].append(Vmg)
        iteration_table_data["Vml"].append(Vml)
        
        pressure = pressure + 0.01
    
    iteration_table = pd.DataFrame(iteration_table_data)
    display(iteration_table)
    
    display(HTML(f"<h2 style='color:green'>FINAL ITERATION RESULT</h2>"))
    display(iteration_table.tail(1).transpose())
    
    
# utility methods
def convert_f_to_r(temp):
    return temp + 459.67

def calculate_tr(temp, temp_c):
    converted_temp = convert_f_to_r(temp)
    converted_temp_c = convert_f_to_r(temp_c)
    return converted_temp / converted_temp_c

def calculate_b(temp_c, pressure_c):
    converted_temp_c = convert_f_to_r(temp_c)
    return (0.07780 * ((10.732 * converted_temp_c) / (pressure_c)))

def calculate_ac(temp_c, pressure_c):
    converted_temp_c = convert_f_to_r(temp_c)
    return (0.45724 * ((10.732 ** 2)* (converted_temp_c ** 2)) / pressure_c)

def calculate_alpha(w, temp, temp_c):
    tr = calculate_tr(temp, temp_c)
    alpha_half = 1 + (0.37464 + (1.54226 * w) - (0.26992 * (w ** 2))) * (1 - (tr ** 0.5))
    return alpha_half ** 2

def calculate_at(ac, alpha):
    return ac * alpha

# Trial section functions
def calculate_A(aT, trial_pressure, sys_temp):
    temp = convert_f_to_r(sys_temp)
    return (aT * trial_pressure) / ((10.732 ** 2) * (temp ** 2))

def calculate_B(b, trial_pressure, sys_temp):
    temp = convert_f_to_r(sys_temp)
    return (b * trial_pressure) / (10.732 * temp)

def cubic_root_calculation(A, B):
    # define coefficents
    a = 1
    b = -(1 - B)
    c = (A - (2*B) - (3*(B ** 2)))
    d = -((A*B) - (B**2) - (B**3))
    
    coefs = [a, b, c, d]
    roots = np.roots(coefs)
    # check for complex numbera and get the real part
    new_roots = []
    for number in roots:
        new_roots.append(number.real)
    
    return new_roots

def get_highest_and_lowest_root(roots):
    try:
        if type(roots) != list:
            raise Exception('The roots have to be a list')
        else:
            lowest = min(roots)
            highest = max(roots)
            return { "zg": highest, "zl": lowest }
    except Exception as error:
        text = f"<div style='color:red'>{error}</div>"
        display(HTML(text))
            

def solve_for_fg_fl(z, B, A, trial_pressure):
    z = z
    B = B
    A = A
    p = trial_pressure
    first_part = math.log(z - B)
    second_part = (A / ((2**1.5)*B))
    numerator = (z + (((2**0.5) + 1) * B))
    denominator = (z - (((2**0.5) - 1) * B))
    log_part = math.log(numerator / denominator)
    f_value = math.exp(z - 1 - first_part - (second_part * log_part)) * trial_pressure
    
    return f_value
    

    
    
display(title)
trap_return = widgets.interact(on_value_change, sys_temp=temperature, specie=specie_dropdown, initial_trial_pressure=pressure_widget)

HTML(value='<h2>Select the species and input the temperature</h2>')

interactive(children=(FloatText(value=104.0, description='Farenheit'), Dropdown(description='Specie', index=1,…